# Detección de equipos de protección personal con Amazon Rekognition

In [ ]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ExifTags, ImageColor
import json
import cv2
import math
import os
import io

In [ ]:
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
bucketName = "{REPLACE_YOUR_BUCKET}"
imageName = "media/ppe-image-group.jpg"

In [ ]:
s3.upload_file(
    Filename="resources/ppe-image-group.jpg",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

In [ ]:
detectPPEResponse = rekognition.detect_protective_equipment(
        Image={
            'S3Object': {
                'Bucket': bucketName,
                'Name': imageName
            }
        },
        SummarizationAttributes={
            'MinConfidence': 90,
            'RequiredEquipmentTypes': [
                'FACE_COVER',
                'HEAD_COVER',
                'HAND_COVER',
            ]
        }
    )

In [ ]:
display(detectPPEResponse)

In [ ]:
detectPPEResponse = rekognition.detect_protective_equipment(
        Image={
            'S3Object': {
                'Bucket': bucketName,
                'Name': imageName
            }
        },
        SummarizationAttributes={
            'MinConfidence': 90,
            'RequiredEquipmentTypes': [
                'FACE_COVER',
                'HEAD_COVER',
                #'HAND_COVER',
            ]
        }
    )

In [ ]:
display(detectPPEResponse)

In [ ]:
def detect_ppe(sourceImage, confidence):

    fill_green='#00d400'
    fill_red='#ff0000'
    fill_yellow='#ffff00'
    line_width=3
    
    imageLocation = 'resources/'+os.path.basename(sourceImage)

    image = Image.open(open(imageLocation,'rb'))
    stream = io.BytesIO()
    image.save(stream, format=image.format)    
    image_binary = stream.getvalue()
    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
    

    for person in detectPPEResponse['Persons']:
        
        found_mask=False

        for body_part in person['BodyParts']:
            ppe_items = body_part['EquipmentDetections']
                 
            for ppe_item in ppe_items:
                if ppe_item['Type'] == 'FACE_COVER':
                    fill_color=fill_green
                    found_mask=True
                    if ppe_item['CoversBodyPart']['Value'] == False:
                        fill_color=fill='#ff0000'
                    box = ppe_item['BoundingBox']
                    left = imgWidth * box['Left']
                    top = imgHeight * box['Top']
                    width = imgWidth * box['Width']
                    height = imgHeight * box['Height']
                    points = (
                            (left,top),
                            (left + width, top),
                            (left + width, top + height),
                            (left , top + height),
                            (left, top)
                        )
                    draw.line(points, fill=fill_color, width=line_width)
     
                    if ppe_item['CoversBodyPart']['Confidence'] < confidence:
                        offset=line_width+ line_width 
                        points = (
                                    (left+offset,top + offset),
                                    (left + width-offset, top+offset),
                                    ((left) + (width-offset), (top-offset) + (height)),
                                    (left+ offset , (top) + (height -offset)),
                                    (left + offset, top + offset)
                                )
                        draw.line(points, fill=fill_yellow, width=line_width)
                
        if found_mask==False:
            box = person['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']
            points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top)
                )
            draw.line(points, fill=fill_red, width=line_width)

    display(image)

In [ ]:
confidence=80 # confidence if the bodypart is covered by the particular PPE 
detect_ppe(imageName, confidence)